In [1]:
import pandas as pd 
df = pd.read_hdf('../data/processed/train/total.hdf')
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,year,weekday,...,to_new_year_days,is_feb_special_time,feb_special_gaussian,from_feb_special_days,to_feb_special_days,is_nov_special_time,nov_special_gaussian,from_nov_special_days,to_nov_special_days,is_holiday
2882335,2015-10-23,33,45,13315,649.0,1.0,23,10,2015,4,...,69,False,0.0,243,122,False,1.755688e-63,17,348,False
2882336,2015-10-05,33,45,13880,229.0,1.0,5,10,2015,0,...,87,False,0.0,225,140,True,0.000000e+00,364,1,False
2882337,2015-10-02,33,45,13881,659.0,1.0,2,10,2015,4,...,90,False,0.0,222,143,False,0.000000e+00,361,4,False
2882338,2015-10-12,33,45,13881,659.0,1.0,12,10,2015,0,...,80,False,0.0,232,133,False,1.522998e-08,6,359,False
2882339,2015-10-04,33,45,13923,169.0,1.0,4,10,2015,6,...,88,False,0.0,224,141,True,0.000000e+00,363,2,False


In [3]:
df.shape

(2935849, 24)

In [4]:
df.shape[0] / df[['shop_id', 'item_id']].drop_duplicates().shape[0]

6.922147768105554

In [5]:
df['date'].max() - df['date'].min()

Timedelta('1033 days 00:00:00')

In [2]:

df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'].astype(str) + '-1')

/home/piotrek/.local/lib/python3.5/site-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [88]:
import numpy as np
df_test = pd.DataFrame([[1,2],[2, np.nan], [3,4]])
df_test.apply(lambda x: np.asarray([[1,2]]), axis=1, raw=True)

Exception: Data must be 1-dimensional

In [71]:
df.groupby(['shop_id', 'item_id'])['date'].max().value_counts()

2015-10-01    31531
2015-09-01    17923
2013-02-01    16473
2015-08-01    15616
2013-12-01    15524
2013-01-01    15325
2014-12-01    15196
2013-03-01    14641
2015-07-01    13570
2015-01-01    12535
2015-03-01    12400
2013-06-01    12245
2013-07-01    12205
2013-08-01    12041
2013-04-01    11845
2014-01-01    11820
2015-02-01    11275
2013-05-01    11073
2014-03-01    11058
2013-09-01    10840
2013-10-01    10731
2014-08-01    10524
2013-11-01    10439
2014-07-01    10312
2014-06-01    10291
2015-06-01     9991
2014-10-01     9959
2014-02-01     9938
2014-11-01     9894
2014-05-01     9886
2014-04-01     9604
2015-05-01     9351
2015-04-01     9273
2014-09-01     8795
Name: date, dtype: int64

In [82]:
pd.Series(pd.date_range(df['date'].min() - pd.DateOffset(months=1), df['date'].max(), freq='M') + pd.DateOffset(days=1)).dt.year

0     2013
1     2013
2     2013
3     2013
4     2013
5     2013
6     2013
7     2013
8     2013
9     2013
10    2013
11    2013
12    2014
13    2014
14    2014
15    2014
16    2014
17    2014
18    2014
19    2014
20    2014
21    2014
22    2014
23    2014
24    2015
25    2015
26    2015
27    2015
28    2015
29    2015
30    2015
31    2015
32    2015
33    2015
dtype: int64

In [62]:
df_gb = df.groupby(['shop_id', 'item_id'])['date'].agg(['min', 'max'])
def range_index(row):
    r = pd.date_range(row['min'] - pd.DateOffset(months=1), row['max'], freq='M') + pd.DateOffset(days=1)
    return pd.Series({'range': r})


arr = []
for name, row in df_gb.iterrows():
    for date in pd.date_range(row['min'] - pd.DateOffset(months=1), row['max'], freq='M') + pd.DateOffset(days=1):
        arr.append(list(name) + [date.year, date.month])
arr = np.asarray(arr)
arr
        

array([[    0,    30,  2013,     2],
       [    0,    31,  2013,     2],
       [    0,    32,  2013,     1],
       ...,
       [   59, 22167,  2014,     4],
       [   59, 22167,  2014,     5],
       [   59, 22167,  2014,     6]])

In [63]:
arr.shape

(3195230, 4)

In [65]:
df_gb = df.groupby(['shop_id', 'item_id', 'year', 'month'])['item_cnt_day'].sum()
print(df_gb.shape)
df_reindexed = df_gb.reindex(index=arr.T.tolist())
print('nulls: ', df_gb.isnull().sum().sum(), df_reindexed.isnull().sum().sum())
print(df_reindexed.shape)

(1609124,)
nulls:  0 1586106
(3195230,)


In [68]:
pd.Series([1,2,3]).rename('a')

0    1
1    2
2    3
Name: a, dtype: int64

In [66]:
df_gb

shop_id  item_id  year  month
0        30       2013  2        31.0
         31       2013  2        11.0
         32       2013  1         6.0
                        2        10.0
         33       2013  1         3.0
                        2         3.0
         35       2013  1         1.0
                        2        14.0
         36       2013  2         1.0
         40       2013  2         1.0
         42       2013  2         1.0
         43       2013  1         1.0
         49       2013  2         2.0
         51       2013  1         2.0
                        2         3.0
         57       2013  2         1.0
         59       2013  2         1.0
         61       2013  1         1.0
         75       2013  1         1.0
         85       2013  2         1.0
         88       2013  1         1.0
         95       2013  1         1.0
         96       2013  1         1.0
         97       2013  2         1.0
         98       2013  1        25.0
                    

In [11]:
(df_gb['min'] == df_gb['max']).mean()

0.345726721430525

In [14]:
df_item_gb = df.groupby(['item_id'])['date']
(df_item_gb.min() == df_item_gb.max()).mean()

0.13752464804879166

In [15]:
df_item_gb = df_item_gb.agg(['min', 'max'])
df_item_gb['shops'] = df.groupby(['item_id'])['shop_id'].nunique()
df_item_gb['one_date'] = (df_item_gb['min'] == df_item_gb['max'])

In [18]:
((df_item_gb['shops'] == 1) & (df_item_gb['one_date'])).mean()

0.11560508093731371

In [102]:
pd.DataFrame(df_gb.iloc[:10]['min'].map(lambda x: [1,2,3]))

min
shop_id item_id           
0       30       [1, 2, 3]
        31       [1, 2, 3]
        32       [1, 2, 3]
        33       [1, 2, 3]
        35       [1, 2, 3]
        36       [1, 2, 3]
        40       [1, 2, 3]
        42       [1, 2, 3]
        43       [1, 2, 3]
        49       [1, 2, 3]

In [55]:
pd.date_range(df_gb['min'],df_gb['max'], freq='M')

TypeError: Cannot convert input [shop_id  item_id
0        30        2013-02-01
         31        2013-02-01
         32        2013-01-01
         33        2013-01-01
         35        2013-01-01
         36        2013-02-01
         40        2013-02-01
         42        2013-02-01
         43        2013-01-01
         49        2013-02-01
         51        2013-01-01
         57        2013-02-01
         59        2013-02-01
         61        2013-01-01
         75        2013-01-01
         85        2013-02-01
         88        2013-01-01
         95        2013-01-01
         96        2013-01-01
         97        2013-02-01
         98        2013-01-01
         111       2013-01-01
         115       2013-02-01
         136       2013-02-01
         149       2013-01-01
         151       2013-01-01
         152       2013-02-01
         153       2013-01-01
         198       2013-01-01
         199       2013-02-01
                      ...    
59       22079     2

In [21]:
df.groupby(['shop_id', 'item_id'])['date'].min().dt.year.value_counts()

2013    260663
2014    107083
2015     56378
Name: date, dtype: int64

In [9]:
df[['shop_id', 'item_id']].drop_duplicates().shape[0]

424124

In [11]:
df['date']

2882335   2015-10-01
2882336   2015-10-01
2882337   2015-10-01
2882338   2015-10-01
2882339   2015-10-01
2882340   2015-10-01
2882341   2015-10-01
2882342   2015-10-01
2882343   2015-10-01
2882344   2015-10-01
2882345   2015-10-01
2882346   2015-10-01
2882347   2015-10-01
2882348   2015-10-01
2882349   2015-10-01
2882350   2015-10-01
2882351   2015-10-01
2882352   2015-10-01
2882353   2015-10-01
2882354   2015-10-01
2882355   2015-10-01
2882356   2015-10-01
2882357   2015-10-01
2882358   2015-10-01
2882359   2015-10-01
2882360   2015-10-01
2882361   2015-10-01
2882362   2015-10-01
2882363   2015-10-01
2882364   2015-10-01
             ...    
2882305   2015-09-01
2882306   2015-09-01
2882307   2015-09-01
2882308   2015-09-01
2882309   2015-09-01
2882310   2015-09-01
2882311   2015-09-01
2882312   2015-09-01
2882313   2015-09-01
2882314   2015-09-01
2882315   2015-09-01
2882316   2015-09-01
2882317   2015-09-01
2882318   2015-09-01
2882319   2015-09-01
2882320   2015-09-01
2882321   201

In [24]:
df.as_matrix()

array([[Timestamp('2015-10-01 00:00:00'), 33, 45, ..., 17, 348, False],
       [Timestamp('2015-10-01 00:00:00'), 33, 45, ..., 364, 1, False],
       [Timestamp('2015-10-01 00:00:00'), 33, 45, ..., 361, 4, False],
       ...,
       [Timestamp('2015-09-01 00:00:00'), 32, 25, ..., 359, 6, False],
       [Timestamp('2015-09-01 00:00:00'), 32, 25, ..., 337, 28, False],
       [Timestamp('2015-09-01 00:00:00'), 32, 25, ..., 354, 11, False]],
      dtype=object)

In [27]:

def key_indexes(keys, date_from, date_to):
    r_df = pd.DataFrame(pd.date_range(date_from - pd.DateOffset(months=1), date_to, freq='M') + pd.DateOffset(days=1))
    r_df['year'], r_df['month'] = r_df[0].dt.year, r_df[0].dt.month
    for key_name, key in keys.items():
        r_df[key_name] = key
    return r_df.drop(0, axis=1)

In [36]:
#df = key_indexes({'shop_id': 15, 'item_id': 10}, df['date'].min(), df['date'].max())
df.reindex(columns=df.columns.tolist()  +['a', 'b'], fill_value=0)

,year,month,item_id,shop_id,a,b
0,2013,1,10,15,0,0
1,2013,2,10,15,0,0
2,2013,3,10,15,0,0
3,2013,4,10,15,0,0
4,2013,5,10,15,0,0
5,2013,6,10,15,0,0
6,2013,7,10,15,0,0
7,2013,8,10,15,0,0
8,2013,9,10,15,0,0
9,2013,10,10,15,0,0


In [37]:
df = pd.DataFrame([[0, 5], [1,6], [3,8]])
df

,0,1
0,0,5
1,1,6
2,3,8


In [40]:
df

,1
0,
0,5
1,6
3,8


In [44]:
df.reindex(index=[0,1,2,3], method='ffill')

,1
0,
0,5
1,6
2,6
3,8


In [14]:
range= pd.DataFrame(pd.date_range(df['date'].min(), df['date'].max(), freq='M') + pd.DateOffset(days=1))

In [18]:
range['year'], range['month'], range['shop_id'], range['item_id'] = range[0].dt.year, range[0].dt.month, 12, 23
range

,0,month,day,shop_id,item_id,year
0,2013-02-01,2,1,12,23,2013
1,2013-03-01,3,1,12,23,2013
2,2013-04-01,4,1,12,23,2013
3,2013-05-01,5,1,12,23,2013
4,2013-06-01,6,1,12,23,2013
5,2013-07-01,7,1,12,23,2013
6,2013-08-01,8,1,12,23,2013
7,2013-09-01,9,1,12,23,2013
8,2013-10-01,10,1,12,23,2013
9,2013-11-01,11,1,12,23,2013
